In [1]:
import pyspark
sc = pyspark.SparkContext(appName="NYC")
dataRDD = sc.textFile("yellow_tripdata_2014-05.csv",10)
header = dataRDD.first()
dataNoHeadRDD = dataRDD.filter(lambda x: not x == header and not x=="")
data = dataNoHeadRDD.map(lambda l: l.split(",")).filter(lambda l:len(l) == 18).filter(lambda l: float(l[4])>0)

In [12]:
from datetime import datetime
from datetime import timedelta
import pygeohash as g

# g.encode(lat,lon,precision)
PERCISION = 7
def parse(string):
    return datetime.strptime(string, "%Y-%m-%d %H:%M:%S")
data_need = data.map(lambda l: (l[1], l[2], float(l[4]), g.encode(float(l[6]), float(l[5]),PERCISION), 
                                g.encode(float(l[10]),float(l[9]),PERCISION), float(l[12]),float(l[-1]), str((parse(l[2])-parse(l[1])).total_seconds())))
# percision = 7
msg = ["dr5ru4w","dr5ru4x","dr5ru4q","dr5ru4r","dr5ru4n","dr5ru4p","dr5ru68","dr5ru69","dr5ru62","dr5ru63","dr5ru60","dr5ru61"]
data_MSG = data_need.filter(lambda l: l[3] in msg )

In [ ]:
'''
Index:
0: pickup_datetime
1: dropoff_datetime
2: trip_distance
3: pickup_loc
4: dropoff_loc
5: fare_amount
6: total_amount
7: duration
'''

In [14]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.createDataFrame(data_MSG, ["pickup_datetime","dropoff_datetime","trip_distance","pickup_loc","dropoff_loc","fare_amount","total_amount","duration"])                                                                   
df.write.format("com.databricks.spark.csv").options(header="true").save("tripsFromMSG.csv") 

In [17]:
df.show(10)

+-------------------+-------------------+-------------+----------+-----------+-----------+------------+--------+
|    pickup_datetime|   dropoff_datetime|trip_distance|pickup_loc|dropoff_loc|fare_amount|total_amount|duration|
+-------------------+-------------------+-------------+----------+-----------+-----------+------------+--------+
|2014-05-29 13:49:00|2014-05-29 14:19:00|         3.44|   dr5ru60|    dr5rvnc|       19.5|        20.0|  1800.0|
|2014-05-29 22:13:00|2014-05-29 22:20:00|         0.97|   dr5ru4x|    dr5ruh5|        6.5|         8.9|   420.0|
|2014-05-31 01:33:00|2014-05-31 01:39:00|         1.47|   dr5ru60|    dr5rudm|        6.5|         8.0|   360.0|
|2014-05-12 09:52:17|2014-05-12 10:05:17|          1.2|   dr5ru62|    dr5ru8v|        9.5|        12.0|   780.0|
|2014-05-12 07:26:36|2014-05-12 07:39:44|          2.1|   dr5ru60|    dr5ruv0|       10.5|       13.75|   788.0|
|2014-05-12 08:37:03|2014-05-12 08:43:28|          1.0|   dr5ru63|    dr5ru2s|        6.0|      